In [109]:
# Dependencies and Setup
import pandas as pd


In [110]:
# Load dividends_data.csv and using only the 'ASX Code' and 'Dividend' columns
dividends_data = 'resources/dividends_data.csv'

# Load splits_data.csv and using only the 'Year' and 'Splits' columns
splits_data = 'resources/splits_data.csv'

# Loading combined_data.csv to merge other file into
combined_data = 'resources/merged_industries.csv'

combined_data_df = pd.read_csv(combined_data)

combined_data_df["ASX Code"].count()


20513

In [111]:
# Read files into DataFrame
dividends_data_df = pd.read_csv(dividends_data)
splits_data_df = pd.read_csv(splits_data)


In [112]:
# Sorting by Year & ASX code, then sum totals
total_splits = splits_data_df.groupby(["ASX Code","Year"])["Splits"].sum().reset_index()
total_splits["Splits"] = total_splits["Splits"] + 1

# Converting  Date column to datetime type in dividends_data then extracting Year for Data Column
dividends_data_df["Date"] = pd.to_datetime(dividends_data_df["Date"],dayfirst=True)
dividends_data_df['Year'] = dividends_data_df["Date"].apply(lambda x: x.year - 1 if x.month <= 6 else x.year)
dividends_data_df["Year"] = dividends_data_df["Date"].dt.year

# Removing date column & grouping by ASX code & year, then totaling dividends
dividends_data_df = dividends_data_df.drop(columns=["Date"])
sum_dividends = dividends_data_df.groupby(["ASX Code","Year"])["Dividends"].sum().reset_index()
dividends_data_df.head()

,ASX Code,Dividends,Year
0,AAA,0.12,2024
1,AAA,0.14,2024
2,AAA,0.13,2024
3,AAA,0.12,2024
4,AAA,0.13,2023


In [113]:
# Merge dividends_data with combined_data based on 'ASX Code' and 'Year'.
combined_data = pd.merge(total_splits, sum_dividends, on=["ASX Code", "Year"], how='outer')

# Merging combined_data & dividends_data by Year & ASX Code
combined_dividends_splits_data = pd.merge(total_splits, sum_dividends, on=["ASX Code", "Year"], how='outer')

# Merging combined_dividends_splits_data &  by Year & ASX Code
Final_ASX_data = pd.merge(combined_data_df, combined_dividends_splits_data, on=["ASX Code", "Year"], how='left')
Final_ASX_data.head()

# Removing null values
Final_ASX_data["Splits"].fillna(1, inplace=True)
Final_ASX_data["Dividends"].fillna(0, inplace=True)

# Creating finalised data as CSV.file
Final_ASX_data.to_csv('resources\Cleaned_Final_ASX_data.csv', index=False)
Final_ASX_data["ASX Code"].count()

20513